# 第八章

os.chdir("D:\\TSBOOKDATA\\Chap8") #改变工作目录 

### 例8.1

cpi_data = np.loadtxt("nzcpi.txt")  
index = pd.date_range(start="1970-03",end="2012-06",freq="Q")  
cpi_df = pd.Series(cpi_data,index=index)  
cpi_diff = cpi_df.diff().dropna()  

fig, ax = plt.subplots(1, figsize=(12, 4), dpi=150)  
ax.plot(cpi_diff**2,linestyle="-", color="green")  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
plt.xlabel(xlabel="时间",fontsize=17)  
plt.ylabel(ylabel="残差平方",fontsize=17)  
plt.tight_layout(); plt.savefig("fig/8_1.png")  

### 例8.2

exrate_data = pd.read_csv("ex_rate.csv")["exchangerate"].values  
exrate_df = pd.Series(exrate_data).dropna()  
exrate_diff = exrate_df.diff()**2  

fig = plt.figure(figsize=(12, 6), dpi=150)  
ax1 = fig.add_subplot(211)  
ax1.plot(exrate_df, linestyle="-", color="green")  
plt.xlabel(xlabel="时间", fontsize=17)  
plt.ylabel(ylabel="汇率的对数", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
ax2 = fig.add_subplot(212)  
ax2.plot(exrate_diff, linestyle="-", color="blue")  
plt.xlabel(xlabel="时间", fontsize=17)  
plt.ylabel(ylabel="回报率", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
plt.tight_layout(); plt.savefig("fig/8_2.png")  

### 例8.3

earn_data = pd.read_csv("earnings.csv", index_col="date")["rate"].values  
earn_df = pd.Series(earn_data).interpolate()  

fig, ax = plt.subplots(1, figsize=(12, 4), dpi=150)  
ax.plot(earn_df,linestyle="-", color="green")  
plt.xlabel(xlabel="时间", fontsize=17)  
plt.ylabel(ylabel="十年期美国国债收益率", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
plt.tight_layout(); plt.savefig("fig/8_3.png")  

earn_log = np.log(earn_df)  
earn_logdiff = earn_log.diff().dropna()  

fig = plt.figure(figsize=(12,4),dpi=150)  
ax1 = fig.add_subplot(121)  
ax1.plot(earn_log, linestyle="-", color="green")  
plt.xlabel(xlabel="时间", fontsize=17)  
plt.ylabel(ylabel="收益率的对数", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
ax2 = fig.add_subplot(122)  
ax2.plot(earn_logdiff, linestyle="-", color="blue")  
plt.xlabel(xlabel="时间", fontsize=17)  
plt.ylabel(ylabel="一阶差分", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
plt.tight_layout(); plt.savefig("fig/8_4.png")  

acorr_ljungbox(earn_logdiff,lags = [6,12], boxpierce=True, return_df=True)  

### 例8.4

index_data = pd.read_csv("income_index.csv", usecols=["BAMLHE00EHYITRIV"])  
index_diff = index_data.diff().dropna()  

fig, ax = plt.subplots(1, figsize=(12, 4), dpi=150)  
ax.plot(index_diff**2, linestyle="-", color="green")  
plt.xlabel(xlabel="时间", fontsize=17)  
plt.ylabel(ylabel="一阶差分的平方", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
plt.tight_layout(); plt.savefig("fig/8_5.png")  

from statsmodels.stats.diagnostic import acorr_lm  

for n in (1,2,3,4,5):  
&ensp;&ensp;&ensp;&ensp;alm = acorr_lm(index_diff,nlags=n)[0:2]  
&ensp;&ensp;&ensp;&ensp;print(alm)  

acorr_ljungbox(index_diff**2,lags = [1,2,3,4,5],boxpierce=True,return_df=True)  

from arch.univariate import arch_model  

am = arch_model(index_diff,mean='Zero',vol='ARCH',p=2).fit(disp="off")  
print(am.summary().tables[1])  

### 例8.5

cufer = pd.read_csv("CUFER.csv",usecols=["DEXCHUS"]).interpolate()  
ut = cufer.diff().dropna()  

fig = plt.figure(figsize=(12,4), dpi=150)  
ax1 = fig.add_subplot(121)  
ax1.plot(cufer,linestyle="-",color='blue')  
ax1.set_ylabel(ylabel="中/美外汇汇率 ", fontsize=17)  
ax1.set_xlabel(xlabel="时间",fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
ax2 = fig.add_subplot(122)  
ax2.plot(ut,linestyle="-", color='green')  
ax2.set_ylabel(ylabel="一阶差分", fontsize=17)  
ax2.set_xlabel(xlabel="时间",fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
fig.tight_layout(); plt.savefig(fname='fig/8_6.png')  

fig = plt.figure(figsize=(12,4), dpi=150)  
ax1 = fig.add_subplot(121)  
ACF(ut, lag=31)  
ax2 = fig.add_subplot(122)  
PACF(ut, lag=31, xlabel='lag', fname='fig/8_7.png')  

ut_est = ARIMA(ut,trend='n', order=(3,0,0)).fit()  
print(ut_est.summary().tables[1])  

vt = ut_est.resid  
acorr_ljungbox(vt, lags = [2,4], boxpierce=True, return_df=True)  

acorr_ljungbox(vt**2, lags = [2,4], boxpierce=True, return_df=True)  

vt_model = arch_model(vt, mean='Zero', vol="GARCH").fit(disp="off")  
print(vt_model.summary().tables[1])  

xgmf = vt_model.conditional_volatility  
ypsw = vt/(np.sqrt(xgmf))  
acorr_ljungbox(ypsw, lags = [2,4,6,8], boxpierce=True, return_df=True)  

acorr_ljungbox(ypsw**2, lags = [2,4,6,8], boxpierce=True, return_df=True)  

import scipy.stats as st  

fig = plt.figure(figsize=(12, 4),dpi=150)  
ax1 = fig.add_subplot(121)  
res1 = st.probplot(ypsw, plot=ax1)  
ax1.set_ylabel(ylabel="Ordered Values", fontsize=17)  
ax1.set_xlabel(xlabel="Theoretical quantiles", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
ax2 = fig.add_subplot(122)  
ax2.hist(ypsw, bins=360)  
ax2.set_title('histogram', fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
plt.tight_layout(); plt.savefig("fig/8_8.png")  

def self_JBtest(y):    
&ensp;&ensp;&ensp;&ensp;n = y.size   
&ensp;&ensp;&ensp;&ensp;ym = y - y.mean()    
&ensp;&ensp;&ensp;&ensp;M2 = np.mean(ym \** 2)    
&ensp;&ensp;&ensp;&ensp;skew =  np.mean(ym \** 3)/M2 \** 1.5      
&ensp;&ensp;&ensp;&ensp;krut = np.mean(ym \** 4)/M2 \** 2    
&ensp;&ensp;&ensp;&ensp;JB = n \* (skew \** 2/6 + (krut-3 ) \** 2/24)  
&ensp;&ensp;&ensp;&ensp;pvalue = 1 - st.chi2.cdf(JB,df=2)  
&ensp;&ensp;&ensp;&ensp;print("偏度：",st.skew(y),skew)  
&ensp;&ensp;&ensp;&ensp;print("峰值：",st.kurtosis(y)+3,krut)  
&ensp;&ensp;&ensp;&ensp;print("JB检验：",st.jarque_bera(y))  
&ensp;&ensp;&ensp;&ensp;return np.array([JB,pvalue])  

print(self_JBtest(ypsw))  